using titanic.csv:

Load the data and observe some features
Break into a random 80/20 train test split
Fit a decision tree using the independent variables 'pclass + sex + age + sibsp' and the dependent variable 'survived'
Plot the full tree
Makre sure 'survived' is a qualitatie variable taking 1 (yes) or 0 (no) 

Print the performance measures of the full model:
    In-sample accuracy
    Out-of-sample accuracy
Defined by: 
    In-sample percent survivors correctly predicted (on training set)
    In-sample percent fatalities correctly predicted (on training set)
    Out-of-sample percent survivors correctly predicted (on test set)
    Out-of-sample percent fatalities correctly predicted (on test set)
    
Use cross-validation to find the best parameters to prune the tree. 
Plot a graph with 'tree-size' as the x-axis and 'number of misclassifications' as the y-axis. 
Find the optimum tree size based off of the misclassification rate

Prune the tree to the optimal size. Plot the pruned tree

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.data as tfds

from tqdm import tqdm

### Load the dataframe from Titanic.csv and explore

In [2]:
datapath: str = os.path.join('data', 'Titanic.csv')

In [3]:
df: pd.DataFrame = pd.read_csv(datapath)

In [4]:
df.head()

,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [5]:
df.describe()

,Unnamed: 0,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,655.000000,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,378.020061,0.486055,14.413500,1.041658,0.865560,51.758669,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,328.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,655.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,982.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,1309.000000,1.000000,80.000000,8.000000,9.000000,512.329224,328.000000


In [6]:
df = df.drop(["Unnamed: 0", "name", "parch", "fare", "body", "embarked", "boat", "home.dest", "cabin", "ticket"], axis=1)
df.head()

,pclass,survived,sex,age,sibsp
0,1st,1,female,29.0000,0
1,1st,1,male,0.9167,1
2,1st,0,female,2.0000,1
3,1st,0,male,30.0000,1
4,1st,0,female,25.0000,1


In [7]:
df.isna().sum()

pclass        0
survived      0
sex           0
age         263
sibsp         0
dtype: int64

In [8]:
df['age'].fillna(value=df['age'].mean(), inplace=True)

In [9]:
df.isna().sum()

pclass      0
survived    0
sex         0
age         0
sibsp       0
dtype: int64

In [10]:
df['sex'] = df['sex'].astype('category')
sex_categories = df['sex'].cat.categories.tolist()
df['pclass'] = df['pclass'].astype('category')
pclass_categories = df['pclass'].cat.categories.tolist()
df['sex'] = df['sex'].cat.codes
df['pclass'] = df['pclass'].cat.codes

In [11]:
df.head()

,pclass,survived,sex,age,sibsp
0,0,1,0,29.0000,0
1,0,1,1,0.9167,1
2,0,0,0,2.0000,1
3,0,0,1,30.0000,1
4,0,0,0,25.0000,1


In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int8   
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   int8   
 3   age       1309 non-null   float64
 4   sibsp     1309 non-null   int64  
dtypes: float64(1), int64(2), int8(2)
memory usage: 33.4 KB
None


In [13]:
def prepare(df, target, test_percent, shuffle=True):
    assert 0 < test_percent < 1.
    
    num_test = round(len(df) * test_percent)
    y = df.loc[:, df.columns == target]
    x = df.loc[:, df.columns != target]

    dataset = tfds.Dataset.from_tensor_slices((x.values, y.values))
    
    if shuffle: 
        dataset = dataset.shuffle(1000)
        
    test = dataset.take(num_test)
    train = dataset.skip(num_test)
    
    return train, test

In [14]:
train, test = prepare(df, 'survived', 0.2, shuffle=True)